In [25]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_excel
import seaborn as sns

In [26]:
df = pd.read_excel('airline_countries.xls')
df.head()

,Country,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14
0,Ireland,Aer Lingus,320906734,2,0,0,0,0,0
1,Russia,Aeroflot*,1197672318,76,14,128,6,1,88
2,Argentina,Aerolineas Argentinas,385803648,6,0,0,1,0,0
3,Mexico,Aeromexico*,596871813,3,1,64,5,0,0
4,Canada,Air Canada,1865253802,2,0,0,2,0,0


In [27]:
import pycountry 
# create country codes to use geospatial data in Tableau
def alpha3code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_3
           # .alpha_3 means 3-letter country code 
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE
# create a column for country code 
df['CODE']=alpha3code(df.Country)
df.head()

,Country,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14,CODE
0,Ireland,Aer Lingus,320906734,2,0,0,0,0,0,IRL
1,Russia,Aeroflot*,1197672318,76,14,128,6,1,88,None
2,Argentina,Aerolineas Argentinas,385803648,6,0,0,1,0,0,ARG
3,Mexico,Aeromexico*,596871813,3,1,64,5,0,0,MEX
4,Canada,Air Canada,1865253802,2,0,0,2,0,0,CAN


In [28]:
import geopandas
# first merge geopandas data with my data
# 'naturalearth_lowres' is geopandas dataset
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# rename the columns so that I can merge with my data
world.columns=['pop_est', 'continent', 'name', 'CODE', 'gdp_md_est', 'geometry']
#merge with my data 
merge=pd.merge(world,df,on='CODE')
merge.head()

,pop_est,continent,name,CODE,gdp_md_est,geometry,Country,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14
0,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",Canada,Air Canada,1865253802,2,0,0,2,0,0
1,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",United States,Alaska Airlines*,965346773,5,0,0,5,1,88
2,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",United States,American*,5228357340,21,5,101,17,3,416
3,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",United States,Delta / Northwest*,6525658894,24,12,407,24,2,51
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",United States,Hawaiian Airlines,493877795,0,0,0,1,0,0


In [29]:
# group by continent and save as csv to use in Tableau
grouped_cont = merge.groupby('continent').sum()
grouped_cont.to_csv('by_continent.csv', index=True)
merge.to_csv('by_continent_full.csv', index=True)

In [30]:
# number weeks/year = 52.1429
# conversion factor from km to mile = 0.62137
# divided by 1 million
df['mil_miles_year_per_seat'] = (df['avail_seat_km_per_week'] * 52.1429 * 0.62137) / 1000000

In [31]:
# took mean number of fatalities over 14 year periods divided 
# by the number of miles flown
df['fatality_per_mil_mile_85_99'] = (df['fatalities_85_99']/14) / df['mil_miles_year_per_seat']
df['fatality_per_mil_mile_00_14'] = (df['fatalities_00_14']/14) / df['mil_miles_year_per_seat']
df.head()

,Country,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14,CODE,mil_miles_year_per_seat,fatality_per_mil_mile_85_99,fatality_per_mil_mile_00_14
0,Ireland,Aer Lingus,320906734,2,0,0,0,0,0,IRL,10397.389020,0.000000,0.000000
1,Russia,Aeroflot*,1197672318,76,14,128,6,1,88,None,38804.623552,0.000236,0.000162
2,Argentina,Aerolineas Argentinas,385803648,6,0,0,1,0,0,ARG,12500.051225,0.000000,0.000000
3,Mexico,Aeromexico*,596871813,3,1,64,5,0,0,MEX,19338.666899,0.000236,0.000000
4,Canada,Air Canada,1865253802,2,0,0,2,0,0,CAN,60434.286180,0.000000,0.000000


In [32]:
mean_mile_85 = (df['fatality_per_mil_mile_85_99'].mean())
mean_mile_00 = (df['fatality_per_mil_mile_00_14'].mean())

In [33]:
lst = ['Between 1985 and 1999', 'Between 2000 and 2014'] 
  
# list of int 
lst2 = [mean_mile_85, mean_mile_00] 
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
fatal_by_mile = pd.DataFrame(list(zip(lst, lst2)), 
               columns =['Timeframe', 'Fatalities per Million Miles']) 
fatal_by_mile

,Timeframe,Fatalities per Million Miles
0,Between 1985 and 1999,0.000286
1,Between 2000 and 2014,0.000143


In [34]:
# convert to csv format to use in Tableau
fatal_by_mile.to_csv('fatal_mile.csv', index=False)

In [35]:
# filter for U.S. data only
airlines_us_df = df[(df.Country == 'United States')]
airlines_us_df.head()

,Country,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14,CODE,mil_miles_year_per_seat,fatality_per_mil_mile_85_99,fatality_per_mil_mile_00_14
8,United States,Alaska Airlines*,965346773,5,0,0,5,1,88,USA,31277.268048,0.000000,0.000201
11,United States,American*,5228357340,21,5,101,17,3,416,USA,169398.954393,0.000043,0.000175
19,United States,Delta / Northwest*,6525658894,24,12,407,24,2,51,USA,211431.568557,0.000137,0.000017
26,United States,Hawaiian Airlines,493877795,0,0,0,1,0,0,USA,16001.657238,0.000000,0.000000
43,United States,Southwest Airlines,3276525770,1,0,0,8,0,0,USA,106159.545606,0.000000,0.000000


In [36]:
# average number of fatalities/year between 2000-2014
us_each00 = (airlines_us_df['fatalities_00_14'].sum())/14

In [37]:
# link to population info
# https://www.multpl.com/united-states-population/table/by-year
us_pop_07 = 301230000
us_pop_17 = 325100000
# women of childbearing age (15-49)
# https://www.who.int/data/maternal-newborn-child-adolescent-ageing/indicator-explorer-new/mca/women-of-reproductive-age-(15-49-years)-population-(thousands)
us_pop_18_childbearing = 74774530

In [38]:
# used 2007 as midpoint data for flight data ranging from 2000-14
odds_us_fly_death = us_each00 / us_pop_07
odds_us_fly_death

1.629035241225262e-07

In [39]:
# lightening data average in US between 2009-2018
# https://www.weather.gov/safety/lightning-odds
odds_lightening = 1/1222000
odds_lightening

8.183306055646482e-07

In [40]:
# https://www.statista.com/statistics/527321/deaths-due-to-choking-in-the-us/
odds_choking = 5216 / us_pop_17
odds_choking

1.6044294063365117e-05

In [41]:
# https://www.cdc.gov/nchs/data/nvsr/nvsr69/nvsr69-13-508.pdf
odds_childbirth = 658 /us_pop_18_childbearing
odds_childbirth

8.79978784219707e-06

In [42]:
# link to death by fire 
# https://www.usfa.fema.gov/data/statistics/#tab-2
odds_fire = 3400 / us_pop_17
odds_fire

1.0458320516764072e-05

In [43]:
lst = ['Airplane Crash', 'Lightening Strike', 'Choking', 'Maternal Death in Childbirth', 'Burned in Fire'] 
  
# list of int 
lst2 = [odds_us_fly_death, odds_lightening, odds_choking, odds_childbirth, odds_fire] 
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
compare_df = pd.DataFrame(list(zip(lst, lst2)), 
               columns =['Death due to', 'Likelihood per person']) 
compare_df.to_csv('death_odds.csv', index=False)
compare_df 

,Death due to,Likelihood per person
0,Airplane Crash,1.629035e-07
1,Lightening Strike,8.183306e-07
2,Choking,1.604429e-05
3,Maternal Death in Childbirth,8.799788e-06
4,Burned in Fire,1.045832e-05


In [44]:
cars_data = pd.read_csv('table_02_17_121019.csv')  
cars_data.head()

,Table 2-17: Motor Vehicle Safety Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,NaN,1960,1965,1970,1975,1980,1985,1990,1991,1992,...,2009,2010,2011,2012,2013,2014,2015,2016,(R) 2017,2018
1,Fatalities,"36,399","47,089","52,627","44,525","51,091","43,825","44,599","41,508","39,250",...,"33,883","32,999","32,479","33,782","32,893","32,744","35,484","37,806","37,473","36,560"
2,Injured persons,N,N,N,N,N,N,"(R) 3,246,000","(R) 3,107,000","(R) 3,079,000",...,"(R) 2,224,000","(R) 2,248,000","(R) 2,227,000","(R) 2,369,000","(R) 2,319,000","(R) 2,343,000","(R) 2,455,000","(R) 3,062,000","2,745,000","2,710,000"
3,Crashes,N,N,N,N,N,N,"6,471,000","6,117,000","6,000,000",...,"5,505,000","5,419,000","5,338,000","5,615,000","5,687,000","6,064,000","6,296,000","6,821,000","6,453,000","6,734,000"
4,Vehicle-miles (millions),"718,763","887,811","1,109,724","1,327,664","1,527,295","1,774,826","2,144,362","2,172,050","2,247,151",...,"2,956,764","2,967,266","2,950,402","2,969,433","2,988,280","3,025,656","3,095,373","3,174,408","3,212,347","3,240,327"


In [45]:
# rename columns
cars_data.columns = cars_data.iloc[0]
cars_data = cars_data[1:5]
cars_data.head()

,NaN,1960,1965,1970,1975,1980,1985,1990,1991,1992,...,2009,2010,2011,2012,2013,2014,2015,2016,(R) 2017,2018
1,Fatalities,"36,399","47,089","52,627","44,525","51,091","43,825","44,599","41,508","39,250",...,"33,883","32,999","32,479","33,782","32,893","32,744","35,484","37,806","37,473","36,560"
2,Injured persons,N,N,N,N,N,N,"(R) 3,246,000","(R) 3,107,000","(R) 3,079,000",...,"(R) 2,224,000","(R) 2,248,000","(R) 2,227,000","(R) 2,369,000","(R) 2,319,000","(R) 2,343,000","(R) 2,455,000","(R) 3,062,000","2,745,000","2,710,000"
3,Crashes,N,N,N,N,N,N,"6,471,000","6,117,000","6,000,000",...,"5,505,000","5,419,000","5,338,000","5,615,000","5,687,000","6,064,000","6,296,000","6,821,000","6,453,000","6,734,000"
4,Vehicle-miles (millions),"718,763","887,811","1,109,724","1,327,664","1,527,295","1,774,826","2,144,362","2,172,050","2,247,151",...,"2,956,764","2,967,266","2,950,402","2,969,433","2,988,280","3,025,656","3,095,373","3,174,408","3,212,347","3,240,327"


In [46]:
cars_data.set_index(cars_data.columns[0], inplace=True)
cars_data = cars_data.T
cars_data.head()

In [48]:
cars_data = cars_data.replace(',','', regex=True)
cars_data['Injured persons'] = cars_data['Injured persons'].str.replace(r"\(.*\)","")
cars_data.head(10)

nan,Fatalities,Injured persons,Crashes,Vehicle-miles (millions)
0,,,,
1960,36399,N,N,718763
1965,47089,N,N,887811
1970,52627,N,N,1109724
1975,44525,N,N,1327664
1980,51091,N,N,1527295
1985,43825,N,N,1774826
1990,44599,3246000,6471000,2144362
1991,41508,3107000,6117000,2172050
1992,39250,3079000,6000000,2247151


In [49]:
# remove rows with missing data
cars_data = cars_data[6:]
cars_data.head()

nan,Fatalities,Injured persons,Crashes,Vehicle-miles (millions)
0,,,,
1990,44599,3246000,6471000,2144362
1991,41508,3107000,6117000,2172050
1992,39250,3079000,6000000,2247151
1993,40150,3163000,6106000,2296378
1994,40716,3275000,6496000,2357588


In [50]:
cars_data['Vehicle-miles (millions)'] = cars_data['Vehicle-miles (millions)'].astype(int)
cars_data['Fatalities'] = cars_data['Fatalities'].astype(int)
cars_data['Injured persons'] = cars_data['Injured persons'].astype(int)

In [51]:
cars_data['car_fatality_per_mil_mile'] = cars_data['Fatalities'] / cars_data['Vehicle-miles (millions)']
cars_data['car_injury_per_mil_mile'] = cars_data['Injured persons'] / cars_data['Vehicle-miles (millions)']
cars_data.head()

nan,Fatalities,Injured persons,Crashes,Vehicle-miles (millions),car_fatality_per_mil_mile,car_injury_per_mil_mile
0,,,,,,
1990,44599,3246000,6471000,2144362,0.020798,1.513737
1991,41508,3107000,6117000,2172050,0.019110,1.430446
1992,39250,3079000,6000000,2247151,0.017467,1.370179
1993,40150,3163000,6106000,2296378,0.017484,1.377386
1994,40716,3275000,6496000,2357588,0.017270,1.389132


In [52]:
car_mile = cars_data[['car_fatality_per_mil_mile', 'car_injury_per_mil_mile']]

In [53]:
fly_data = pd.read_csv('better_airplane_data.csv') 
fly_data.head()

,Table 2-9: U.S. Air Carriera Safety Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,NaN,1960,1965,1970,1975,1980,1985,1990,1991,1992,...,2009,2010,2011,2012,2013,(R) 2014,(R) 2015,(R) 2016,(R) 2017,(P) 2018
1,Total fatalities,499,261,146,124,1,526,39,50,33,...,52,2,0,0,9,0,0,0,0,1
2,Total seriously injured persons,N,N,107,81,19,30,29,26,22,...,26,17,21,18,9,14,24,18,19,25
3,Total accidents,90,83,55,37,19,21,24,26,18,...,30,30,33,26,23,31,29,30,32,30
4,Fatal accidents,17,9,8,3,1,7,6,4,4,...,2,1,0,0,2,0,0,0,0,1


In [54]:
fly_data.columns = fly_data.iloc[0]
fly_data = fly_data[1:6]
fly_data.head()

,NaN,1960,1965,1970,1975,1980,1985,1990,1991,1992,...,2009,2010,2011,2012,2013,(R) 2014,(R) 2015,(R) 2016,(R) 2017,(P) 2018
1,Total fatalities,499,261,146,124,1,526,39,50,33,...,52,2,0,0,9,0,0,0,0,1
2,Total seriously injured persons,N,N,107,81,19,30,29,26,22,...,26,17,21,18,9,14,24,18,19,25
3,Total accidents,90,83,55,37,19,21,24,26,18,...,30,30,33,26,23,31,29,30,32,30
4,Fatal accidents,17,9,8,3,1,7,6,4,4,...,2,1,0,0,2,0,0,0,0,1
5,Aircraft-miles (millions),"1,130","1,536","2,685","2,478","2,924","3,631","4,948","4,825","5,039",...,"7,466","7,598","7,714","7,660","7,673","7,691","7,822","8,017","8,155","8,474"


In [55]:
fly_data.set_index(fly_data.columns[0], inplace=True)
fly_data = fly_data.T
fly_data.head()

nan,Total fatalities,Total seriously injured persons,Total accidents,Fatal accidents,Aircraft-miles (millions)
0,,,,,
1960,499,N,90,17,"1,130"
1965,261,N,83,9,"1,536"
1970,146,107,55,8,"2,685"
1975,124,81,37,3,"2,478"
1980,1,19,19,1,"2,924"


In [56]:
fly_data = fly_data.replace(',','', regex=True)
fly_data['Total fatalities'] = fly_data['Total fatalities'].str.replace(r"\(.*\)","")
fly_data['Total seriously injured persons'] = fly_data['Total seriously injured persons'].str.replace(r"\(.*\)","")

fly_data.head()

nan,Total fatalities,Total seriously injured persons,Total accidents,Fatal accidents,Aircraft-miles (millions)
0,,,,,
1960,499,N,90,17,1130
1965,261,N,83,9,1536
1970,146,107,55,8,2685
1975,124,81,37,3,2478
1980,1,19,19,1,2924


In [57]:
fly_data = fly_data[6:]
fly_data.head()

nan,Total fatalities,Total seriously injured persons,Total accidents,Fatal accidents,Aircraft-miles (millions)
0,,,,,
1990,39,29,24,6,4948
1991,50,26,26,4,4825
1992,33,22,18,4,5039
1993,1,19,23,1,5249
1994,239,31,23,4,5478


In [58]:
fly_data['Fatal accidents'] = fly_data['Fatal accidents'].astype(int)
fly_data['Aircraft-miles (millions)'] = fly_data['Aircraft-miles (millions)'].astype(int)
fly_data['Total accidents'] = fly_data['Total accidents'].astype(int)
fly_data.to_csv('fly_data.csv', index=True)

In [59]:
fly_data['fatal_per_mil_mile'] = fly_data['Fatal accidents'] / fly_data['Aircraft-miles (millions)']
fly_data['injury_per_mil_mile'] = fly_data['Total accidents'] / fly_data['Aircraft-miles (millions)']

In [60]:
air_mile = fly_data[['fatal_per_mil_mile', 'injury_per_mil_mile']]

In [61]:
both_mile = car_mile.merge(air_mile, left_index=True, right_index=True)

In [62]:
both_mile.to_csv('by_mile.csv', index=True)

In [63]:
total_fly_fatal = fly_data[['Total fatalities', 'Total seriously injured persons']]
total_car_fatal = cars_data[['Fatalities', 'Injured persons']]

In [64]:
total_fatal = total_fly_fatal.merge(total_car_fatal, left_index=True, right_index=True)

In [228]:
total_fatal.to_csv('total_fatal.csv', index=True)